# Validators

Obsplus provides a way to declare and test data requirements. The implementation is specifically geared towards nested tree structures (like obspy's `Catalog` object), but can work for any type of object. 

<div class="alert alert-warning">

**Warning**: This is a fairly advanced feature of ObsPlus intended primarily for library authors and users with stringent data requirements. The built-in validators will meet most people's needs.  
</div>

<div class="alert alert-warning">

**Warning**: In the future we may move much of this functionality to obspy as described in [this proposal](https://github.com/obspy/obspy/issues/2154), but an appropriate deprecation cycle will be implemented. 
</div>


## Built-in Validators
Obsplus comes with a few built-in validators. See the [catalog validation](catalog_validation.ipynb) page for more details.

## Custom Validators
Here is an example which creates a custom validator for ensuring a group of events all have at least four picks and that the origins have latitude and longitude defined. We will use the namespace `"_silly_test"` to let obsplus know these validators should be grouped together.  


In [1]:
import obsplus
import obspy
import obspy.core.event as ev
from obsplus.validate import validator, validate

namespace = '_silly_test'

/home/derrick_chambers/anaconda3/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}
/home/derrick_chambers/anaconda3/lib/python3.7/site-packages/distributed/config.py:20: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  defaults = yaml.load(f)


In [2]:
# We simply have to decorate a callable with the `validator` decorator
# and specify the class it is to act on and its namespace
@validator(namespace, ev.Event)
def ensure_events_have_four_picks(event):
    picks = event.picks
    assert len(picks) >= 4

    
@validator(namespace, ev.Origin)
def ensure_preferred_origin_set(origin):
    assert origin.latitude is not None
    assert origin.longitude is not None
    

Now we create an event we know will violate both conditions and run the `validate` function. it should raise an `AssertionError`.

In [3]:
cat = obspy.read_events()
for event in cat:
    event.picks = []
    for origin in event.origins:
        origin.latitude = None
        origin.longitude = None

In [4]:
try:
    validate(cat, namespace)
except AssertionError:
    print('catalog failed validations')

catalog failed validations


But we could also return a report of failures (in the form of a dataframe)

In [5]:
report = validate(cat, namespace, report=True)

In [6]:
report

,message,object,passed,validator
0,validator ensure_events_have_four_picks failed...,"[resource_id, event_type, event_type_certainty...",False,ensure_events_have_four_picks
1,validator ensure_events_have_four_picks failed...,"[resource_id, event_type, event_type_certainty...",False,ensure_events_have_four_picks
2,validator ensure_events_have_four_picks failed...,"[resource_id, event_type, event_type_certainty...",False,ensure_events_have_four_picks
3,validator ensure_preferred_origin_set failed o...,"[resource_id, time, time_errors, longitude, lo...",False,ensure_preferred_origin_set
4,validator ensure_preferred_origin_set failed o...,"[resource_id, time, time_errors, longitude, lo...",False,ensure_preferred_origin_set
5,validator ensure_preferred_origin_set failed o...,"[resource_id, time, time_errors, longitude, lo...",False,ensure_preferred_origin_set
